In [68]:
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np

# Statsbomb WIP

## Competitions

In [3]:
competitions = pd.read_json('../raw_data/open-data/data/competitions.json')
# adding match count to competitions
for i, row in competitions.iterrows():
    with open(f"../raw_data/open-data/data/matches/{row['competition_id']}/{row['season_id']}.json") as file:
        json_file = json.load(file)
    
    # print(row['competition_id'], row['season_id'], row['competition_name'], row['season_name'], len(json_file))
    competitions.loc[i, 'match_count'] = len(json_file)

In [4]:
feats_to_see = ['competition_id', 'season_id', 'country_name', 'competition_name', 'season_name', 'match_count']
comp_20p_matches = competitions.query('match_count >= 20')[feats_to_see]
comp_20p_matches

,competition_id,season_id,country_name,competition_name,season_name,match_count
0,9,27,Germany,1. Bundesliga,2015/2016,306.0
22,37,90,England,FA Women's Super League,2020/2021,131.0
23,37,42,England,FA Women's Super League,2019/2020,87.0
24,37,4,England,FA Women's Super League,2018/2019,108.0
26,43,106,International,FIFA World Cup,2022,64.0
27,43,3,International,FIFA World Cup,2018,64.0
34,1238,108,India,Indian Super league,2021/2022,115.0
35,11,90,Spain,La Liga,2020/2021,35.0
36,11,42,Spain,La Liga,2019/2020,33.0
37,11,4,Spain,La Liga,2018/2019,34.0


In [5]:
competitions_id = comp_20p_matches.query('match_count > 38')[['competition_id', 'season_id']].reset_index(drop=True)
competitions_id.loc[len(competitions_id)] = [53, 106]
competitions_id

,competition_id,season_id
0,9,27
1,37,90
2,37,42
3,37,4
4,43,106
5,43,3
6,1238,108
7,11,27
8,7,27
9,2,27


In [6]:
competitions_id.to_csv('../processed_data/statsbomb/competitions_id.csv', index=False, header=True)

## Matches

In [7]:
def get_clean_match(df):
    '''
    This function receives a match dataframe and returns it without uninteresting
    columns. It adds the home and away team ids as columns and the target, ie, 1
    if home wins, 0 if it's a draw and -1 if away wins.
    '''
    cols_to_drop = ['match_status', 'match_status_360', 'last_updated', 'last_updated_360', 'metadata', 'stadium', 'referee']
    df = df.drop(columns=cols_to_drop)
    df.loc[:, 'home_id'] = df.loc[:, 'home_team'].map(lambda x: x.get('home_team_id'))
    df.loc[:, 'away_id'] = df.loc[:, 'away_team'].map(lambda x: x.get('away_team_id'))
    df.loc[:, 'target'] = df.apply(lambda x: 1 if x['home_score'] > x['away_score'] else 0 if x['home_score'] == x['away_score'] else -1, axis=1)
    return df

In [8]:
match1 = pd.read_json('../raw_data/open-data/data/matches/2/27.json')
match1_clean = get_clean_match(match1)
match1_clean.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage,home_id,away_id,target
0,3754058,2016-01-02,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 22, 'home_team_name': 'Leices...","{'away_team_id': 28, 'away_team_name': 'AFC Bo...",0,0,20,"{'id': 1, 'name': 'Regular Season'}",22,28,0
1,3754245,2015-10-17,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 27, 'home_team_name': 'West B...","{'away_team_id': 41, 'away_team_name': 'Sunder...",1,0,9,"{'id': 1, 'name': 'Regular Season'}",27,41,1
2,3754136,2015-12-19,18:30:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 37, 'home_team_name': 'Newcas...","{'away_team_id': 59, 'away_team_name': 'Aston ...",1,1,17,"{'id': 1, 'name': 'Regular Season'}",37,59,0
3,3754037,2016-04-30,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 29, 'home_team_name': 'Everto...","{'away_team_id': 28, 'away_team_name': 'AFC Bo...",2,1,36,"{'id': 1, 'name': 'Regular Season'}",29,28,1
4,3754039,2016-02-13,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 31, 'home_team_name': 'Crysta...","{'away_team_id': 23, 'away_team_name': 'Watfor...",1,2,26,"{'id': 1, 'name': 'Regular Season'}",31,23,-1


## Events

In [9]:
evs1 = pd.read_json('../raw_data/open-data/data/events/15978.json')
evs1.head(1)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour
0,c49b57c0-8b25-4183-acd7-b13a52464607,1,1,2023-09-06,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.0,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# to add id columns
for feat in ['type', 'team']:
  evs1.loc[:, f'{feat}_id'] = evs1.loc[:, feat].map(lambda x: x['id'])
  evs1.loc[:, f'{feat}_name'] = evs1.loc[:, feat].map(lambda x: x['name'])
  
evs1.head(3)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour,type_id,type_name,team_id,team_name
0,c49b57c0-8b25-4183-acd7-b13a52464607,1,1,2023-09-06,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.0,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,Starting XI,210,Real Sociedad
1,129dbbf1-1faf-48cd-a550-022a5154cb90,2,1,2023-09-06,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",0.0,"{'formation': 433, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,Starting XI,217,Barcelona
2,19a741d4-8122-41b2-8ea6-02769da5ff5d,3,1,2023-09-06,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.0,NaN,[957a518f-23a0-43c7-9b16-fce3911d3e3f],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,Half Start,210,Real Sociedad


In [11]:
evs1.groupby(by=['type_id', 'type_name', 'team_id']).size()

type_id  type_name       team_id
2        Ball Recovery   210         34
                         217         70
3        Dispossessed    210         14
                         217          8
4        Duel            210         38
                         217         27
6        Block           210         21
                         217         13
9        Clearance       210         21
                         217          4
10       Interception    210         21
                         217         10
14       Dribble         210          6
                         217         26
16       Shot            210          8
                         217         12
17       Pressure        210        141
                         217        112
18       Half Start      210          2
                         217          2
19       Substitution    210          3
                         217          3
21       Foul Won        210          8
                         217         19
22     

In [12]:
# pass analysis
pass_outcome = evs1.query('type_id == 30')['pass'].map(lambda x: x.get('outcome'))
pass_outcome.isna().sum()/len(pass_outcome)
for row in pass_outcome[~pass_outcome.isna()].map(lambda x: x.get('name')):
  print(row)
# pass_outcome


Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Out
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Pass Offside
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Unknown
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Out
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Out
Incomplete
Incomplete
Incomplete
Out
Incomplete
Incomplete
Incomplete
Out
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
Incomplete
In

In [13]:
pass_recipient = evs1.query('type_id == 30')['pass'].map(lambda x: x.get('recipient'))
pass_recipient

4       {'id': 5573, 'name': 'Héctor Alfredo Moreno He...
6                                                    None
9            {'id': 6374, 'name': 'Nélson Cabral Semedo'}
12          {'id': 5213, 'name': 'Gerard Piqué Bernabéu'}
15             {'id': 5492, 'name': 'Samuel Yves Umtiti'}
                              ...                        
3884         {'id': 6676, 'name': 'Igor Zubeldia Elorza'}
3886     {'id': 6605, 'name': 'Joseba Zaldúa Bengoetxea'}
3891           {'id': 3042, 'name': 'Mikel Merino Zazón'}
3899                                                 None
3901     {'id': 5203, 'name': 'Sergio Busquets i Burgos'}
Name: pass, Length: 1148, dtype: object

In [14]:
# seems like recipient exists iff pass is completed
print(len(pass_recipient))
(pass_recipient.isna() == ~pass_outcome.isna()).sum()

1148


1061

In [15]:
evs1.loc[:, 'pass']#.map(lambda x: x.get('recipient'))

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4       {'recipient': {'id': 5573, 'name': 'Héctor Alf...
                              ...                        
3900                                                  NaN
3901    {'recipient': {'id': 5203, 'name': 'Sergio Bus...
3902                                                  NaN
3903                                                  NaN
3904                                                  NaN
Name: pass, Length: 3905, dtype: object

In [16]:
passes = evs1.query('type_id == 30')
# passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 1 if x.get('recipient') else 0)
# changed to better see if pass is incomplete if there is outcome (normally incomplete pass)
passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 0 if x.get('outcome') else 1)
passes

/tmp/ipykernel_15448/3736965171.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 0 if x.get('outcome') else 1)


,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour,type_id,type_name,team_id,team_name,pass_completed
4,d652dea1-a7c5-4140-9ae8-f893acd0b72d,5,1,2023-09-06 00:00:00.120,0,0,"{'id': 30, 'name': 'Pass'}",2,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 210, 'name': 'Real Sociedad'}",3.239211,NaN,[1d614951-0549-465a-9a4c-106287f86e64],1.0,"{'id': 6685, 'name': 'Mikel Oyarzabal Ugarte'}","{'id': 12, 'name': 'Right Midfield'}","[60.0, 40.0]","{'recipient': {'id': 5573, 'name': 'Héctor Alf...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,210,Real Sociedad,1
6,13e796c1-812a-4c25-a910-05982c1652df,7,1,2023-09-06 00:00:05.397,0,5,"{'id': 30, 'name': 'Pass'}",2,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 210, 'name': 'Real Sociedad'}",3.740729,NaN,[46187568-527b-4502-be1d-8985a1d4c7c7],NaN,"{'id': 5573, 'name': 'Héctor Alfredo Moreno He...","{'id': 5, 'name': 'Left Center Back'}","[49.9, 39.3]","{'length': 14.723111, 'angle': -1.997986, 'hei...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,210,Real Sociedad,0
9,0fbebb10-09b0-454a-a1e3-79fdeb0a2038,10,1,2023-09-06 00:00:10.315,0,10,"{'id': 30, 'name': 'Pass'}",3,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",1.529271,NaN,[bdca08b8-080b-4623-9b70-842566126ea3],NaN,"{'id': 5492, 'name': 'Samuel Yves Umtiti'}","{'id': 5, 'name': 'Left Center Back'}","[27.3, 23.9]","{'recipient': {'id': 6374, 'name': 'Nélson Cab...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,217,Barcelona,1
12,f99f883b-4973-47f0-a227-0774c4b79545,13,1,2023-09-06 00:00:16.237,0,16,"{'id': 30, 'name': 'Pass'}",3,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",1.871526,NaN,[6a0f55e5-f8bb-4e94-a549-a6f7052a5297],NaN,"{'id': 6374, 'name': 'Nélson Cabral Semedo'}","{'id': 2, 'name': 'Right Back'}","[26.1, 76.4]","{'recipient': {'id': 5213, 'name': 'Gerard Piq...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,217,Barcelona,1
15,d615902f-c8d6-44d1-9819-aaf92d3f3b19,16,1,2023-09-06 00:00:19.368,0,19,"{'id': 30, 'name': 'Pass'}",3,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",2.012964,NaN,[fc6e5557-120a-46ae-8973-be43c1bbbc4a],NaN,"{'id': 5213, 'name': 'Gerard Piqué Bernabéu'}","{'id': 3, 'name': 'Right Center Back'}","[13.6, 54.6]","{'recipient': {'id': 5492, 'name': 'Samuel Yve...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,217,Barcelona,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3884,a775043d-bc6d-448b-9412-3b33b3383658,3885,2,2023-09-06 00:47:12.375,92,12,"{'id': 30, 'name': 'Pass'}",175,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 4, 'name': 'From Throw In'}","{'id': 210, 'name': 'Real Sociedad'}",1.554603,NaN,[0f5253c4-a32c-461e-bb5b-d4f664b30e8a],1.0,"{'id': 6605, 'name': 'Joseba Zaldúa Bengoetxea'}","{'id': 2, 'name': 'Right Back'}","[89.1, 80.0]","{'recipient': {'id': 6676, 'name': 'Igor Zubel...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,210,Real Sociedad,1
3886,28761d2b-bc7e-4dac-9e11-8d6ae6938183,3887,2,2023-09-06 00:47:14.010,92,14,"{'id': 30, 'name': 'Pass'}",175,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 4, 'name': 'From Throw In'}","{'id': 210, 'name': 'Real Sociedad'}",2.426845,NaN,[11ef2120-7a31-41c7-b882-3e75229b4dd1],NaN,"{'id': 6676, 'name': 'Igor Zubeldia Elorza'}

In [17]:
# shot analysis
evs1.query('type_id == 16')['shot'].iloc[0]

{'statsbomb_xg': 0.05368826,
 'end_location': [120.0, 36.5, 1.1],
 'outcome': {'id': 97, 'name': 'Goal'},
 'first_time': True,
 'technique': {'id': 91, 'name': 'Half Volley'},
 'body_part': {'id': 38, 'name': 'Left Foot'},
 'type': {'id': 87, 'name': 'Open Play'},
 'freeze_frame': [{'location': [114.8, 29.3],
   'player': {'id': 5477, 'name': 'Ousmane Dembélé'},
   'position': {'id': 21, 'name': 'Left Wing'},
   'teammate': False},
  {'location': [109.6, 36.0],
   'player': {'id': 5470, 'name': 'Ivan Rakitić'},
   'position': {'id': 10, 'name': 'Center Defensive Midfield'},
   'teammate': False},
  {'location': [113.0, 26.9],
   'player': {'id': 6379, 'name': 'Sergi Roberto Carnicer'},
   'position': {'id': 13, 'name': 'Right Center Midfield'},
   'teammate': False},
  {'location': [109.7, 31.3],
   'player': {'id': 5213, 'name': 'Gerard Piqué Bernabéu'},
   'position': {'id': 3, 'name': 'Right Center Back'},
   'teammate': False},
  {'location': [103.7, 44.7],
   'player': {'id': 5211

In [18]:
shot_on_target_ids = [96, 97, 100]
goal_id = 97
shots = evs1.query('type_id == 16')
shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
shots.loc[:, 'shot_goal'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') == goal_id else 0)
shots.loc[:, 'shot_xg'] = shots.loc[:, 'shot'].map(lambda x: x.get('statsbomb_xg'))

shots

/tmp/ipykernel_15448/1869883104.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
/tmp/ipykernel_15448/1869883104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots.loc[:, 'shot_goal'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') == goal_id else 0)
/tmp/ipykernel_15448/1869883104.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour,type_id,type_name,team_id,team_name,shot_on_target,shot_goal,shot_xg
586,73b4f96a-2a64-4f7c-8fb1-0729c921e99f,587,1,2023-09-06 00:11:54.021,11,54,"{'id': 16, 'name': 'Shot'}",23,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 3, 'name': 'From Free Kick'}","{'id': 210, 'name': 'Real Sociedad'}",0.750514,NaN,[3ff9335f-c213-4f6b-a824-34a247f97df2],NaN,"{'id': 6679, 'name': 'Aritz Elustondo Irribaria'}","{'id': 3, 'name': 'Right Center Back'}","[107.5, 29.7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.05368826, 'end_location': [...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,210,Real Sociedad,1,1,0.053688
728,176d86cf-3674-4f8c-9c06-9b199a10d0a9,729,1,2023-09-06 00:16:26.099,16,26,"{'id': 16, 'name': 'Shot'}",32,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.753255,NaN,[46dceb1f-5dcb-4493-9e0d-240bdb0af5c1],NaN,"{'id': 6695, 'name': 'Juan Miguel Jiménez López'}","{'id': 24, 'name': 'Left Center Forward'}","[99.4, 50.9]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.048043124, 'end_location': ...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,210,Real Sociedad,0,0,0.048043
1595,66da79ef-f087-4d40-805a-b07a280f05aa,1596,1,2023-09-06 00:35:05.307,35,5,"{'id': 16, 'name': 'Shot'}",63,"{'id': 217, 'name': 'Barcelona'}","{'id': 2, 'name': 'From Corner'}","{'id': 217, 'name': 'Barcelona'}",1.232881,NaN,"[386845a5-f11b-4e70-a826-e73f8065cd90, a7273ca...",NaN,"{'id': 5213, 'name': 'Gerard Piqué Bernabéu'}","{'id': 3, 'name': 'Right Center Back'}","[112.8, 46.7]",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.059001160000000004, 'end_lo...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,217,Barcelona,0,0,0.059001
1810,c9deaf75-3381-49fd-beb2-6e078d0971f5,1811,1,2023-09-06 00:40:51.538,40,51,"{'id': 16, 'name': 'Shot'}",73,"{'id': 217, 'name': 'Barcelona'}","{'id': 2, 'name': 'From Corner'}","{'id': 217, 'name': 'Barcelona'}",0.271398,NaN,[1ac15525-ccbe-4d9b-ac27-68fe4e6f3d04],NaN,"{'id': 5213, 'name': 'Gerard Piqué Bernabéu'}","{'id': 3, 'name': 'Right Center Back'}","[113.1, 35.4]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.14770305, 'end_location': [...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,217,Barcelona,1,0,0.147703
1815,7a635e80-a845-4933-93a1-f810d9c749d6,1816,1,2023-09-06 00:41:57.824,41,57,"{'id': 16, 'name': 'Shot'}",74,"{'id': 217, 'name': 'Barcelona'}","{'id': 2, 'name': 'From Corner'}","{'id': 217, 'name': 'Barcelona'}",1.364274,NaN,"[9e1016b7-11b2-4b0d-8122-b0224009d985, d7753c8...",NaN,"{'id': 5470, 'name': 'Ivan Rakitić'}","{'id': 10, 'name': 'Center Defensive Midfield'}","[112.4, 38.0]",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.100465104, 'end_location': ...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,217,Barcelona,0,0,0.100465
2319,cfceefce-f6b8-49e6-ae55-d3b4fa6950d2,2320,2,2023-09-06 00:08:02.953,53,2,"{'id': 16, 'name': 'Shot'}",97,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 2, 'name': 'From Corner'}","{'id': 210, 'name': 'Real Sociedad'}",1.317323,NaN,[eb45b32c-68f5-4a27-988f-20dda81cf199],NaN,"{'id': 6685, 'name': 'Mikel Oyarzabal Ugarte'}","{'id': 12, 'name': 'Right Midfield'}","[105.4, 34.7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.05929953600000001, 'end_loc...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,210,Real Sociedad,1,0,0.059300
2526,56984762-35f3-4831-ba6a-d66fbe73a59f,2527,2,2023-09-06 00:12:00.585,57,0,"{'id': 16, 'name': 'Shot'}",102,"{'id': 217, 'name': 'Barcelona'}","{'id': 4, 'name': 'From Throw In'}","{'id': 217, 'name': 'Barcelona'}",2.374076,NaN,[94807271-baea-4cab-97b3-def060c66bc5],NaN,"{'id': 5477, 'name': 'Ousmane Dembélé'}","{'id': 21, 'nam

In [19]:
# aggregating passes and shots
passes_grouped = passes.groupby(by='team_id').agg({'pass_completed': 'sum', 'type_id': 'size'})
passes_grouped.loc[:, 'pass_precision'] = passes_grouped.loc[:, 'pass_completed'] / passes_grouped.loc[:, 'type_id']
passes_grouped.rename(columns={'type_id': 'pass_total'}, inplace=True)
passes_grouped


,pass_completed,pass_total,pass_precision
team_id,,,
210,276,365,0.756164
217,681,783,0.869732


In [20]:
shots_grouped = shots.groupby(by='team_id').agg({'shot_on_target': 'sum', 'shot_goal': 'sum', 'shot_xg': 'sum', 'type_id': 'size'})
shots_grouped.loc[:, 'shot_precision'] = shots_grouped.loc[:, 'shot_on_target'] / shots_grouped.loc[:, 'type_id']
shots_grouped.loc[:, 'shot_conversion'] = shots_grouped.loc[:, 'shot_goal'] / shots_grouped.loc[:, 'type_id']
shots_grouped.rename(columns={'type_id': 'shot_total'}, inplace=True)
shots_grouped

,shot_on_target,shot_goal,shot_xg,shot_total,shot_precision,shot_conversion
team_id,,,,,,
210,5,1,0.914412,8,0.625000,0.125000
217,7,2,1.398165,12,0.583333,0.166667


In [21]:
grouped_df = pd.concat([passes_grouped, shots_grouped], axis=1)
grouped_df

,pass_completed,pass_total,pass_precision,shot_on_target,shot_goal,shot_xg,shot_total,shot_precision,shot_conversion
team_id,,,,,,,,,
210,276,365,0.756164,5,1,0.914412,8,0.625000,0.125000
217,681,783,0.869732,7,2,1.398165,12,0.583333,0.166667


In [22]:
shots_grouped.to_dict()

{'shot_on_target': {210: 5, 217: 7},
 'shot_goal': {210: 1, 217: 2},
 'shot_xg': {210: 0.914412035, 217: 1.398165227},
 'shot_total': {210: 8, 217: 12},
 'shot_precision': {210: 0.625, 217: 0.5833333333333334},
 'shot_conversion': {210: 0.125, 217: 0.16666666666666666}}

### The function that does the magic

In [23]:
def get_events_info(match_id, home_id, away_id):
  '''
  This function receives a certain match id, looks for the events dataframe
  and returns a dictionary whose keys are the teams and the values are dictionaries
  with grouped information (shots and passes)
  '''
  evs = pd.read_json(f'../raw_data/open-data/data/events/{match_id}.json')
  evs.loc[:, 'type_id'] = evs.loc[:, 'type'].map(lambda x: x['id'])
  evs.loc[:, 'team_id'] = evs.loc[:, 'team'].map(lambda x: x['id'])
  # passes analysis
  pass_id = 30
  passes = evs.query(f'type_id == {pass_id}')
  # determines if a pass is completed if it doesn't have an output (which exists
  # only if the pass is not completed)
  passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 0 if x.get('outcome') else 1)
  # shots analysis
  shot_on_target_ids = [96, 97, 100]
  goal_id = 97
  shot_id = 16
  shots = evs.query(f'type_id == {shot_id}')
  # determines if the shot was on target, a goal and adds its xG
  shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
  shots.loc[:, 'shot_goal'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') == goal_id else 0)
  shots.loc[:, 'shot_xg'] = shots.loc[:, 'shot'].map(lambda x: x.get('statsbomb_xg'))
  # aggregating the stats
  passes_grouped = passes.groupby(by='team_id').agg({'pass_completed': 'sum', 'type_id': 'size'})
  passes_grouped.loc[:, 'pass_precision'] = passes_grouped.loc[:, 'pass_completed'] / passes_grouped.loc[:, 'type_id'].map(lambda x: max(1,x))
  passes_grouped.rename(columns={'type_id': 'pass_total'}, inplace=True)
  
  shots_grouped = shots.groupby(by='team_id').agg({'shot_on_target': 'sum', 'shot_goal': 'sum', 'shot_xg': 'sum', 'type_id': 'size'})
  shots_grouped.loc[:, 'shot_precision'] = shots_grouped.loc[:, 'shot_on_target'] / shots_grouped.loc[:, 'type_id'].map(lambda x: max(1,x))
  shots_grouped.loc[:, 'shot_conversion'] = shots_grouped.loc[:, 'shot_goal'] / shots_grouped.loc[:, 'type_id'].map(lambda x: max(1,x))
  shots_grouped.rename(columns={'type_id': 'shot_total'}, inplace=True)
  
  grouped_df = pd.concat([passes_grouped, shots_grouped], axis=1)
  
  dummy_df = pd.DataFrame({'match_id': [match_id],
                           'home_id': [home_id],
                           'away_id': [away_id]})
  
  home_df = dummy_df.merge(grouped_df, left_on='home_id', right_on='team_id', suffixes=('', '_home'))
  away_df = home_df.merge(grouped_df, left_on='away_id', right_on='team_id', suffixes=('_home', '_away'))
  
  return away_df.drop(columns=['home_id', 'away_id'])
  

## Testing 1

In [24]:
comp_test = competitions_id.iloc[0]
comp_test

competition_id     9
season_id         27
Name: 0, dtype: int64

In [25]:
comp_id = comp_test['competition_id']
seas_id = comp_test['season_id']
match_test = pd.read_json(f'../raw_data/open-data/data/matches/{comp_id}/{seas_id}.json')
match_test = get_clean_match(match_test)
match_test.head(2)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage,home_id,away_id,target
0,3890561,2016-05-14,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 175, 'home_team_name': 'Hoffe...","{'away_team_id': 181, 'away_team_name': 'Schal...",1,4,34,"{'id': 1, 'name': 'Regular Season'}",175,181,-1
1,3890505,2016-04-02,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 169, 'home_team_name': 'Bayer...","{'away_team_id': 184, 'away_team_name': 'Eintr...",1,0,28,"{'id': 1, 'name': 'Regular Season'}",169,184,1


In [26]:
ev_info_1 = get_events_info(match_test.iloc[0]['match_id'], match_test.iloc[0]['home_id'], match_test.iloc[0]['away_id'])
ev_info_1

/tmp/ipykernel_15448/2654161342.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 0 if x.get('outcome') else 1)
/tmp/ipykernel_15448/2654161342.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
/tmp/ipykernel_15448/2654161342.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,match_id,pass_completed_home,pass_total_home,pass_precision_home,shot_on_target_home,shot_goal_home,shot_xg_home,shot_total_home,shot_precision_home,shot_conversion_home,pass_completed_away,pass_total_away,pass_precision_away,shot_on_target_away,shot_goal_away,shot_xg_away,shot_total_away,shot_precision_away,shot_conversion_away
0,3890561,422,506,0.833992,8,1,1.168875,13,0.615385,0.076923,515,590,0.872881,9,3,1.605704,12,0.75,0.25


In [27]:
events = []
for _, row in match_test.iterrows():
    match_id = row['match_id']
    home_id = row['home_id']
    away_id = row['away_id']
    events.append(get_events_info(match_id, home_id, away_id))

/tmp/ipykernel_15448/2654161342.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 0 if x.get('outcome') else 1)
/tmp/ipykernel_15448/2654161342.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
/tmp/ipykernel_15448/2654161342.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [28]:
events_test = pd.concat(events, axis=0).reset_index(drop=True)
events_test.head(10)

,match_id,pass_completed_home,pass_total_home,pass_precision_home,shot_on_target_home,shot_goal_home,shot_xg_home,shot_total_home,shot_precision_home,shot_conversion_home,pass_completed_away,pass_total_away,pass_precision_away,shot_on_target_away,shot_goal_away,shot_xg_away,shot_total_away,shot_precision_away,shot_conversion_away
0,3890561,422,506,0.833992,8,1,1.168875,13,0.615385,0.076923,515,590,0.872881,9,3,1.605704,12,0.750000,0.250000
1,3890505,598,698,0.856734,11,1,1.837664,20,0.550000,0.050000,213,316,0.674051,0,0,0.121696,4,0.000000,0.000000
2,3890511,504,606,0.831683,11,2,1.496190,12,0.916667,0.166667,255,342,0.745614,6,2,0.800510,12,0.500000,0.166667
3,3890515,490,620,0.790323,3,1,0.795829,10,0.300000,0.100000,154,280,0.550000,5,2,0.833502,9,0.555556,0.222222
4,3890411,531,638,0.832288,5,2,1.399013,7,0.714286,0.285714,328,419,0.782816,2,0,0.216394,5,0.400000,0.000000
5,3890397,308,405,0.760494,7,1,1.378773,14,0.500000,0.071429,404,499,0.809619,8,0,1.456773,19,0.421053,0.000000
6,3890401,790,894,0.883669,15,4,5.312835,24,0.625000,0.166667,194,278,0.697842,2,1,0.185625,3,0.666667,0.333333
7,3890396,481,610,0.788525,16,1,1.780996,20,0.800000,0.050000,256,371,0.690027,5,1,0.483422,5,1.000000,0.200000
8,3890384,627,758,0.827177,10,3,1.816692,14,0.714286,0.214286,199,323,0.616099,7,1,0.968795,8,0.875000,0.125000
9,3890385,392,491,0.798371,16,3,2.770773,26,0.615385,0.115385,282,394,0.715736,4,1,0.442469,7,0.571429,0.142857


In [29]:
match_test = match_test.merge(events_test, on='match_id', how='left')
match_test.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage,home_id,away_id,target,pass_completed_home,pass_total_home,pass_precision_home,shot_on_target_home,shot_goal_home,shot_xg_home,shot_total_home,shot_precision_home,shot_conversion_home,pass_completed_away,pass_total_away,pass_precision_away,shot_on_target_away,shot_goal_away,shot_xg_away,shot_total_away,shot_precision_away,shot_conversion_away
0,3890561,2016-05-14,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 175, 'home_team_name': 'Hoffe...","{'away_team_id': 181, 'away_team_name': 'Schal...",1,4,34,"{'id': 1, 'name': 'Regular Season'}",175,181,-1,422,506,0.833992,8,1,1.168875,13,0.615385,0.076923,515,590,0.872881,9,3,1.605704,12,0.750000,0.250000
1,3890505,2016-04-02,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 169, 'home_team_name': 'Bayer...","{'away_team_id': 184, 'away_team_name': 'Eintr...",1,0,28,"{'id': 1, 'name': 'Regular Season'}",169,184,1,598,698,0.856734,11,1,1.837664,20,0.550000,0.050000,213,316,0.674051,0,0,0.121696,4,0.000000,0.000000
2,3890511,2016-04-08,20:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 173, 'home_team_name': 'Herth...","{'away_team_id': 178, 'away_team_name': 'Hanno...",2,2,29,"{'id': 1, 'name': 'Regular Season'}",173,178,0,504,606,0.831683,11,2,1.496190,12,0.916667,0.166667,255,342,0.745614,6,2,0.800510,12,0.500000,0.166667
3,3890515,2016-04-09,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 171, 'home_team_name': 'Hambu...","{'away_team_id': 872, 'away_team_name': 'Darms...",1,2,29,"{'id': 1, 'name': 'Regular Season'}",171,872,-1,490,620,0.790323,3,1,0.795829,10,0.300000,0.100000,154,280,0.550000,5,2,0.833502,9,0.555556,0.222222
4,3890411,2015-12-20,16:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 173, 'home_team_name': 'Herth...","{'away_team_id': 177, 'away_team_name': 'FSV M...",2,0,17,"{'id': 1, 'name': 'Regular Season'}",173,177,1,531,638,0.832288,5,2,1.399013,7,0.714286,0.285714,328,419,0.782816,2,0,0.216394,5,0.400000,0.000000


## Get full season df

In [30]:
def get_full_season_df(competition_id, season_id):
    '''
    This function receives a competition_id and a season_id and returns a 
    dataframe that contains the full info for all the matches.
    '''
    match_df = pd.read_json(f'../raw_data/open-data/data/matches/{competition_id}/{season_id}.json')
    match_df = get_clean_match(match_df)
    
    events = []
    for _, row in match_df.iterrows():
        match_id = row['match_id']
        home_id = row['home_id']
        away_id = row['away_id']
        events.append(get_events_info(match_id, home_id, away_id))
        
    events_df = pd.concat(events, axis=0).reset_index(drop=True)
    match_full_df = match_df.merge(events_df, on='match_id', how='left')
    
    return match_full_df

In [31]:
match_full_test = get_full_season_df(comp_id, seas_id)
match_full_test.head()

/tmp/ipykernel_15448/2654161342.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 0 if x.get('outcome') else 1)
/tmp/ipykernel_15448/2654161342.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
/tmp/ipykernel_15448/2654161342.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage,home_id,away_id,target,pass_completed_home,pass_total_home,pass_precision_home,shot_on_target_home,shot_goal_home,shot_xg_home,shot_total_home,shot_precision_home,shot_conversion_home,pass_completed_away,pass_total_away,pass_precision_away,shot_on_target_away,shot_goal_away,shot_xg_away,shot_total_away,shot_precision_away,shot_conversion_away
0,3890561,2016-05-14,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 175, 'home_team_name': 'Hoffe...","{'away_team_id': 181, 'away_team_name': 'Schal...",1,4,34,"{'id': 1, 'name': 'Regular Season'}",175,181,-1,422,506,0.833992,8,1,1.168875,13,0.615385,0.076923,515,590,0.872881,9,3,1.605704,12,0.750000,0.250000
1,3890505,2016-04-02,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 169, 'home_team_name': 'Bayer...","{'away_team_id': 184, 'away_team_name': 'Eintr...",1,0,28,"{'id': 1, 'name': 'Regular Season'}",169,184,1,598,698,0.856734,11,1,1.837664,20,0.550000,0.050000,213,316,0.674051,0,0,0.121696,4,0.000000,0.000000
2,3890511,2016-04-08,20:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 173, 'home_team_name': 'Herth...","{'away_team_id': 178, 'away_team_name': 'Hanno...",2,2,29,"{'id': 1, 'name': 'Regular Season'}",173,178,0,504,606,0.831683,11,2,1.496190,12,0.916667,0.166667,255,342,0.745614,6,2,0.800510,12,0.500000,0.166667
3,3890515,2016-04-09,15:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 171, 'home_team_name': 'Hambu...","{'away_team_id': 872, 'away_team_name': 'Darms...",1,2,29,"{'id': 1, 'name': 'Regular Season'}",171,872,-1,490,620,0.790323,3,1,0.795829,10,0.300000,0.100000,154,280,0.550000,5,2,0.833502,9,0.555556,0.222222
4,3890411,2015-12-20,16:30:00.000,"{'competition_id': 9, 'country_name': 'Germany...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 173, 'home_team_name': 'Herth...","{'away_team_id': 177, 'away_team_name': 'FSV M...",2,0,17,"{'id': 1, 'name': 'Regular Season'}",173,177,1,531,638,0.832288,5,2,1.399013,7,0.714286,0.285714,328,419,0.782816,2,0,0.216394,5,0.400000,0.000000


In [46]:
to_drop = ['kick_off', 'competition', 'season', 'home_team', 'away_team', 'home_score', 'away_score', 'competition_stage', 'target']
match_full_test_dropped = match_full_test.drop(columns=to_drop)

In [47]:
test_match = match_full_test_dropped.query('match_week == 5').iloc[0]
test_match

match_id                   3890303
match_date              2015-09-20
match_week                       5
home_id                        174
away_id                        181
pass_completed_home            359
pass_total_home                494
pass_precision_home       0.726721
shot_on_target_home             10
shot_goal_home                   0
shot_xg_home              2.906899
shot_total_home                 26
shot_precision_home       0.384615
shot_conversion_home           0.0
pass_completed_away            289
pass_total_away                420
pass_precision_away       0.688095
shot_on_target_away              7
shot_goal_away                   1
shot_xg_away              1.307855
shot_total_away                 10
shot_precision_away            0.7
shot_conversion_away           0.1
Name: 265, dtype: object

In [49]:
# for a given match -> all the matches in which the home team played
all_prev_matches = match_full_test_dropped[(match_full_test.match_date < test_match.match_date) & 
                ((match_full_test_dropped.home_id == test_match.home_id) |
                 (match_full_test_dropped.away_id == test_match.home_id))].sort_values('match_date')
all_prev_matches

,match_id,match_date,match_week,home_id,away_id,pass_completed_home,pass_total_home,pass_precision_home,shot_on_target_home,shot_goal_home,shot_xg_home,shot_total_home,shot_precision_home,shot_conversion_home,pass_completed_away,pass_total_away,pass_precision_away,shot_on_target_away,shot_goal_away,shot_xg_away,shot_total_away,shot_precision_away,shot_conversion_away
298,3890266,2015-08-16,1,174,186,329,454,0.724670,19,1,2.380079,28,0.678571,0.035714,231,351,0.658120,4,3,2.219708,8,0.500000,0.375000
12,3890269,2015-08-22,2,171,174,325,484,0.671488,6,3,1.411376,14,0.428571,0.214286,228,366,0.622951,7,2,1.775539,11,0.636364,0.181818
287,3890279,2015-08-29,3,174,184,324,451,0.718404,10,1,2.406729,14,0.714286,0.071429,298,422,0.706161,4,3,1.815865,6,0.666667,0.500000
278,3890290,2015-09-12,4,173,174,321,439,0.731207,6,2,1.153075,8,0.750000,0.250000,330,441,0.748299,6,1,1.050435,12,0.500000,0.083333


In [54]:
all_prev_matches.drop(columns='match_date').apply('mean', axis=0)

match_id                3.890276e+06
match_week              2.500000e+00
home_id                 1.730000e+02
away_id                 1.795000e+02
pass_completed_home     3.247500e+02
pass_total_home         4.570000e+02
pass_precision_home     7.114420e-01
shot_on_target_home     1.025000e+01
shot_goal_home          1.750000e+00
shot_xg_home            1.837815e+00
shot_total_home         1.600000e+01
shot_precision_home     6.428571e-01
shot_conversion_home    1.428571e-01
pass_completed_away     2.717500e+02
pass_total_away         3.950000e+02
pass_precision_away     6.838827e-01
shot_on_target_away     5.250000e+00
shot_goal_away          2.250000e+00
shot_xg_away            1.715387e+00
shot_total_away         9.250000e+00
shot_precision_away     5.757576e-01
shot_conversion_away    2.850379e-01
dtype: float64

In [56]:
home_cols = [feat for feat in match_full_test_dropped.columns if '_home' in feat]
home_cols

['pass_completed_home',
 'pass_total_home',
 'pass_precision_home',
 'shot_on_target_home',
 'shot_goal_home',
 'shot_xg_home',
 'shot_total_home',
 'shot_precision_home',
 'shot_conversion_home']

In [117]:
def get_past_info_per_team(match_full, team_id, actual_date, if_home, past_games=10):
    home_cols = [feat for feat in match_full.columns if '_home' in feat] 
    away_cols = [feat for feat in match_full.columns if '_away' in feat] 
    home_or_away = home_cols if if_home else away_cols
    all_past_cols = [feat + '_all_past' for feat in home_or_away]
    last_cols = [feat + f'_last_{past_games}' for feat in home_or_away]
 
    all_prev_matches_home = match_full[(match_full.match_date < actual_date) & 
                (match_full.home_id == team_id)][home_cols + ['match_date']]
    all_prev_matches_home.columns = all_past_cols + ['match_date']
    
    all_prev_matches_away = match_full[(match_full.match_date < actual_date) & 
                (match_full.away_id == team_id)][away_cols + ['match_date']]
    all_prev_matches_away.columns = all_past_cols + ['match_date']
    
    all_prev_matches = pd.concat([all_prev_matches_home, all_prev_matches_away]).sort_values('match_date')
    last_prev_matches = all_prev_matches.tail(past_games)
    last_prev_matches.columns = last_cols + ['match_date']
    
    mean_all_prev = all_prev_matches.drop(columns='match_date').apply('mean', axis=0)
    mean_last_prev = last_prev_matches.drop(columns='match_date').apply('mean', axis=0)
    final_df = pd.concat([mean_all_prev, mean_last_prev], axis=0)
    
    return final_df

In [115]:
def get_past_info(match_full, match, past_games=10):
    '''
    Receives a full matches dataset, the match for which we want to compute the 
    mean of the features in the past for each team (home and away)
    and 'past_games' a variable that indicates for how many past matches
    we want to compute the average. It returns a dataframe whose first column
    is the match id, followed by the average of the past metrics for the home 
    and away teams.
    '''
    # TODO: see if we want to drop this columns now or before (while processing)
    to_drop = ['kick_off', 'competition', 'season', 'home_team', 'away_team', 'home_score', 'away_score', 'competition_stage', 'target']
    match_full_dropped = match_full.drop(columns=to_drop)
    match = match.drop(columns=to_drop)
    # useful variables
    actual_date = match['match_date']
    home_team = match['home_id']
    away_team = match['away_id']
    home_last_info = get_past_info_per_team(match_full_dropped, home_team, actual_date, True, past_games)
    away_last_info = get_past_info_per_team(match_full_dropped, away_team, actual_date, False, past_games)
    match_complete = pd.concat([match[['match_id']], home_last_info, away_last_info], axis=0)
    return match_complete
    

In [116]:
get_past_info(match_full_test, test_match)

match_id                          3890303
pass_completed_home_all_past       302.75
pass_total_home_all_past            428.0
pass_precision_home_all_past     0.703581
shot_on_target_home_all_past         10.5
shot_goal_home_all_past              1.25
shot_xg_home_all_past            1.903195
shot_total_home_all_past            16.25
shot_precision_home_all_past     0.632305
shot_conversion_home_all_past    0.093074
pass_completed_home_last_10        302.75
pass_total_home_last_10             428.0
pass_precision_home_last_10      0.703581
shot_on_target_home_last_10          10.5
shot_goal_home_last_10               1.25
shot_xg_home_last_10             1.903195
shot_total_home_last_10             16.25
shot_precision_home_last_10      0.632305
shot_conversion_home_last_10     0.093074
pass_completed_away_all_past       393.75
pass_total_away_all_past            504.0
pass_precision_away_all_past     0.772628
shot_on_target_away_all_past         9.75
shot_goal_away_all_past           